<a href="https://colab.research.google.com/github/debaggi/Junior-Data-Analyst-Cases/blob/main/samokat_tech_junior_da.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Привет!

Всего 3 задания

----------
1 задание
----------

Нужно на основе приложенных выгрузок сделать отчет в Power BI и отправить файл .pbix
(если файл не влезает в почту - положи его на гугл диск и пошарить ссылкой)
Обратно слать выгрузки не нужно.

Требования к отчету - представь, что это все данные, которые вообще есть и ты делаешь первый в компании отчет.
Твоя задача выжать из этих выгрузок максимум пользы.
Какие показатели посчитать, какие визуализации выбрать и как организовать отчет - твой выбор.

Выгрузки:
- warehouses
- products
- orders
- order_lines

В выгрузках рандомные данные, но стоит воспринимать их как фактические.
Всю обработку нужно делать внутри Power BI - исходные файлы оставь как есть.

Бонус:
Если найдешь что-то интересное (аномалии, тренды, зависимости, etc.) - оформи свои мысли в ворде,
сопроводив скринами визуализаций из Power BI и отправь файл .docx
Если хочется сделать в формате презентации или описать внутри Power BI - тоже гуд.

https://pavelbukhtik.notion.site/21d3e6b006284d189514e517d74fa369

In [ ]:
import pandas as pd

In [ ]:
from  google.colab import drive
drive.mount('/content/drive')

In [ ]:
orders = pd.read_csv('/content/drive/MyDrive/Technical Interview/Samokat Tech Data Analytics: Jun.Grade/datasets/orders.csv')
# orders = orders.set_index('warehouse_id')

In [ ]:
warehouses = pd.read_csv('/content/drive/MyDrive/Technical Interview/Samokat Tech Data Analytics: Jun.Grade/datasets/warehouses.csv')

In [ ]:
print('Orders data')
display(orders.head(3))

print('\nWarehouses data')
display(warehouses.head(3))

Цели:
<!-- клиенты -->
- определить количество заказов по каждому даркстору
- определить топ-3 id клиентов, которые наиболее часто осущесвляют заказы
<!-- дарксторы -->
- определить наиболее и наименее часто используемые дарксторы
- определить города, в которых расположены дарксторы из топ-3 дарксторов


In [ ]:
# Внутреннее соединение таблицы orders и таблицы warehouses по атрибуту wirehouse_id
warehouses_orders = pd.merge(orders, warehouses, on='warehouse_id', how='inner')
warehouses_orders['order_date'] = pd.to_datetime(warehouses_orders['order_date'])

print('Сгруппированная таблица заказов и таблица дарксторов по атрибуту wirehouse_id\n')
display(warehouses_orders.sort_values('warehouse_id').head(3))
print(f'\nКол-во записей в таблице: {len(warehouses_orders)}')

# Загружаю датасеты в локальное хранилище

 - warehouses.xlsx
 - orders.xlsx
 - warehouses_orders.xlsx
 - warehouses_orders.csv

In [ ]:
# warehouses_orders.to_csv(('warehouses_orders.csv'))

# Исследование датасетов

## Дарксторы

### Высоко- и низко- нагруженные дарксторы

In [ ]:
warehouses_orders_grouped = warehouses_orders.groupby('warehouse_id').agg(count_order=('order_id', 'count'))
warehouses_orders_grouped = warehouses_orders_grouped.sort_values('count_order', ascending=False)
display(warehouses_orders_grouped)

In [ ]:
# вычисление суммарного числа заказов в разрезе дарксторов
warehouses_top_3_max = list(warehouses_orders_grouped.iloc[:3].index.values)  # топ-3-winners дарксторов по числу заказов
warehouses_top_3_min = list(warehouses_orders_grouped.iloc[-3:].index.values) # топ-3-losers дарксторов по числу заказов

# вычисление суммарного числа заказов по всем дарксторам за все время
total_sum_purchases = warehouses_orders_grouped['count_order'].sum(axis=0)

# процент заказов, который приходится на даркстор
warehouses_orders_grouped['percentage_orders'] = round(warehouses_orders_grouped['count_order'] / total_sum_purchases * 100, 2)
warehouses_orders_grouped.to_excel('warehouses_orders.xlsx', sheet_name='warehouses_orders_grouped')


print(f'Суммарное количество заказов по всем дарксторам сети за все время: {total_sum_purchases}')
print('\n')
print('----------------------------------------------------')
print(f'Дарксторы с максимальным числом заказов: {warehouses_top_3_max}')
print(display(warehouses_orders_grouped.head(3)))
print('\n')
print('----------------------------------------------------')
print(f'Дарксторы с минимальным числом заказов: {warehouses_top_3_min}')
print(display(warehouses_orders_grouped.tail(3)))
print('----------------------------------------------------')

In [ ]:
# информация о топ-3 дарксторах по числу заказов за все время
info_warehouses_top_3_max = warehouses[warehouses['warehouse_id'].isin(warehouses_top_3_max)]
print('Сведения о дарксторах, на которые приходится максимальное число заказов')
display(info_warehouses_top_3_max.set_index('warehouse_id'))
print('--------------------------------------------------------\n')

info_warehouses_top_3_min = warehouses[warehouses['warehouse_id'].isin(warehouses_top_3_min)]
print('Сведения о дарксторах, на которые приходится минимальное число заказов')
display(info_warehouses_top_3_min.set_index('warehouse_id'))

### Анализ клиентов

In [ ]:
# информация о топ-3 покупателях по числу заказов за все время
top_3_clients = pd.DataFrame(warehouses_orders.groupby('user_id')['order_id'].agg('count'))
top_3_clients  = top_3_clients.rename(columns={'order_id':'count_orders'})
top_3_clients = top_3_clients.sort_values('count_orders', ascending=False)

top_3_clients_id = list(top_3_clients.index[:3].values)

display(top_3_clients.head(3))
print(f'Клиенты, входящие к категорию топ-3 клиентов по числу заказов за все время: {top_3_clients_id}')

#### Исследование заказов всех клиентов

In [ ]:
order_lines = pd.read_csv('/content/drive/MyDrive/Technical Interview/Samokat Tech Data Analytics: Jun.Grade/datasets/Order Lines updated.csv')

In [ ]:
# pd.unique(order_lines['price'])

# ['110,25', '45\xa0428,00', '39,00', '36,75', '171,75', '101,25',
      #  ['96,75', '328,50', '119,25', '44,25', '14,25', '18,00', '29,25',
      #  '39,75', '117,00', '21,75', '136,50', '33,75', '60,75', '68,25',
      #  '139,50', '51,75', '168,00', '146,25', '111,75', '89,25', '18,75',
      #  '129,75', '126,75', '37,50', '21,00', '183,00', '59,25', '71,25',
      #  '43,50', '56,25', '73,50', '30,00', '27,00', '42,75', '164,25',
      #  '30,75', '95,25', '118,50', '45\xa0440,00', '179,25', '81,75',
      #  '78,75', '66,75', '79,50', '54,00', '96,00', '141,75', '24,00',
      #  '224,25', '26,25', '41,25', '51,00', '109,50', '63,75', '57,00',
      #  '134,25', '46,50', '65,25', '45\xa0434,00', '104,25', '24,75',
      #  '45\xa0443,00', '52,50', '74,25', '75,00', '45,00', '57,75',
      #  '117,75', '93,00', '156,75', '270,75', '108,75', '45\xa0425,00',
      #  '94,50', '69,00', '12,75', '209,25', '82,50', '63,00', '66,00',
      #  '40,50', '34,50', '128,25', '132,00', '123,00', '67,50', '49,50',
      #  '64,50', '167,25', '116,25', '172,50', '108,00', '33,00', '144,00',
      #  '87,75', '53,25', '42,00', '45\xa0431,00', '155,25', '47,25',
      #  '15,75', '81,00', '27,75', '91,50', '61,50', '153,75', '20,25',
      #  '336,75', '216,75', '99,00', '36,00', '38,25', '17,25', '60,00',
      #  '11,25', '123,75', '183,75', '76,50', '269,25', '62,25', '69,75',
      #  '106,50', '127,50', '103,50', '80,25', '32,25', '55,50', '120,75',
      #  '58,50', '98,25', '93,75', '45\xa0437,00', '72,00', '72,75',
      #  '48,75', '190,50', '210,00', '149,25', '105,75', '636,75', '45,75',
      #  '87,00', '35,25', '284,25', '107,25', '158,25', '202,50', '140,25',
      #  '204,00', '296,25', '144,75', '165,00', '9,75', '254,25', '145,50',
      #  '12,00', '54,75', '92,25', '177,75', '207,00', '85,50', '100,50',
      #  '160,50', '378,75', '177,00', '88,50', '70,50', '50,25', '48,00',
      #  '246,75', '374,25', '84,75', '97,50', '644,25', '135,00', '84,00',
      #  '124,50', '23,25', '113,25', '115,50', '45\xa0422,00', '126,00',
      #  '207,75', '154,50', '231,75', '199,50', '182,25', '125,25',
      #  '203,25', '77,25', '78,00', '163,50', '201,75', '86,25', '186,75',
      #  '194,25', '105,00', '122,25', '83,25', '75,75', '176,25', '171,00',
      #  '152,25', '162,00', '111,00', '135,75', '270,00', '192,75',
      #  '193,50', '138,75', '455,25', '137,25', '150,00', '156,00',
      #  '150,75', '247,50', '273,75', '188,25', '218,25', '161,25',
      #  '90,75', '243,00', '166,50', '90,00', '120,00', '102,75', '234,75',
      #  '114,00', '114,75', '112,50', '235,50', '99,75', '170,25',
      #  '174,00', '148,50', '245,25', '159,00', '180,75', '168,75',
      #  '169,50', '1\xa0094,25', '102,00', '187,50', '456,75', '477,75',
      #  '15,00', '299,25', '524,25', '287,25', '173,25', '159,75',
      #  '147,00', '129,00', '198,00', '142,50', '215,25', '8,25', '151,50',
      #  '201,00', '130,50', '147,75', '234,00', '198,75', '297,00',
      #  '133,50', '511,50', '824,25', '132,75', '261,75', '197,25',
      #  '121,50', '195,75', '186,00', '244,50', '279,75', '1\xa0019,25',
      #  '141,00', '434,25', '711,75', '272,25', '131,25', '216,00',
      #  '205,50', '458,25', '285,00', '480,00', '294,00', '180,00',
      #  '213,75', '178,50', '192,00', '157,50', '219,00', '210,75',
      #  '475,50', '249,75', '191,25', '195,00', '9,00', '6,75', '449,25',
      #  '246,00', '579,75', '379,50', '189,00', '258,75', '143,25',
      #  '226,50', '417,75', '326,25', '405,00', '367,50', '261,00',
      #  '289,50', '5,25', '321,75', '222,00', '669,75', '353,25', '359,25',
      #  '282,75', '262,50', '741,75', '196,50', '584,25', '1\xa0118,25',
      #  '225,00', '45\xa0419,00', '162,75', '252,75', '239,25', '318,00',
      #  '343,50', '242,25', '174,75', '314,25', '237,00', '318,75',
      #  '189,75', '175,50', '185,25', '380,25', '355,50', '221,25',
      #  '288,00', '206,25', '204,75', '474,75', '138,00', '237,75',
      #  '181,50', '283,50', '346,50', '229,50', '266,25', '248,25',
      #  '327,00', '561,75', '304,50', '281,25', '251,25', '217,50',
      #  '256,50', '225,75', '582,00', '260,25', '267,00', '451,50',
      #  '674,25', '249,00', '165,75', '612,00', '337,50', '447,75',
      #  '253,50', '970,50', '411,75', '208,50', '252,00', '223,50',
      #  '398,25', '433,50', '312,00', '294,75', '153,00', '263,25',
      #  '756,00', '348,75', '666,00', '341,25', '345,75', '551,25',
      #  '213,00', '381,75', '231,00', '582,75', '295,50', '219,75',
      #  '276,00', '396,75', '228,75', '419,25', '441,75', '258,00',
      #  '599,25', '200,25', '322,50', '388,50', '276,75', '431,25',
      #  '321,00', '408,00', '277,50', '542,25', '504,75', '2\xa0099,25',
      #  '275,25', '310,50', '267,75', '660,00', '487,50', '538,50',
      #  '483,00', '386,25', '397,50', '288,75', '211,50', '264,75',
      #  '567,00', '390,75', '335,25', '292,50', '607,50', '471,75',
      #  '330,00', '369,00', '1\xa0274,25', '306,75', '606,75', '345,00',
      #  '1\xa0251,75', '364,50', '478,50', '438,00', '220,50', '363,00',
      #  '366,00', '243,75', '573,75', '384,00', '259,50', '230,25',
      #  '307,50', '319,50', '945,75', '631,50', '504,00', '603,00',
      #  '303,75', '474,00', '214,50', '238,50', '615,00', '429,75',
      #  '375,00', '426,75', '752,25', '444,00', '240,75', '971,25',
      #  '495,00', '1\xa0431,75', '212,25', '271,50', '435,00', '313,50',
      #  '241,50', '573,00', '453,75', '464,25', '184,50', '468,75',
      #  '232,50', '1\xa0011,75', '454,50', '273,00', '342,00', '378,00',
      #  '666,75', '994,50', '265,50', '549,75', '279,00', '1\xa0038,00',
      #  '257,25', '315,75', '354,00', '315,00', '702,75', '233,25',
      #  '555,00', '308,25', '416,25', '339,75', '293,25', '373,50',
      #  '618,75', '291,75', '974,25', '358,50', '347,25', '633,00',
      #  '512,25', '406,50', '749,25', '227,25', '710,25', '1\xa0337,25',
      #  '899,25', '-152\xa0184,00', '635,25', '408,75', '461,25', '255,75',
      #  '637,50', '648,00', '572,25', '306,00', '522,00', '1\xa0982,25',
      #  '473,25', '554,25', '125\xa0035,00', '6,00', '395,25', '411,00',
      #  '432,00', '250,50', '556,50', '333,75', '324,75', '598,50',
      #  '382,50', '562,50', '350,25', '588,75', '1\xa0901,25', '442,50',
      #  '592,50', '427,50', '605,25', '349,50', '936,75', '576,75',
      #  '240,00', '601,50', '334,50', '301,50', '344,25', '865,50',
      #  '278,25', '468,00', '410,25', '286,50', '356,25', '403,50',
      #  '566,25', '264,00', '450,75', '329,25', '701,25', '228,00',
      #  '16,50', '291,00', '290,25', '613,50', '1\xa0043,25', '978,00',
      #  '658,50', '325,50', '394,50', '904,50', '567,75', '722,25',
      #  '493,50', '402,00', '807,00', '819,00', '274,50', '1\xa0791,00',
      #  '469,50', '494,25', '1\xa0008,75', '734,25', '1\xa0050,00',
      #  '546,75', '1\xa0359,00', '457,50', '630,75', '13,97', '714,75',
      #  '732,00', '1\xa0124,25', '486,75', '465,00', '742,50', '89,21',
      #  '13,50'], dtype=object)

#### Исследование заказов топ-3 клиентов

In [ ]:
# Фильтрация датафрейма warehouses_orders по атрибуту user_id. Выбираю все заказы, сделанные клиентами из списка top_3_clients
print('Сведения о заказах клиентов из категории топ-3')
print('')

info_orders_top_3_clients = warehouses_orders[warehouses_orders['user_id'].isin(top_3_clients_id)].set_index('user_id').copy()
display(info_orders_top_3_clients)

In [ ]:
order_id_top_3_clients = info_orders_top_3_clients['order_id']

# идентификаторы клиентов из категории топ-3
clients_id_top_3 = list(pd.unique(top_3_clients.index[:3]))

orders_by_client_197929 = info_orders_top_3_clients[info_orders_top_3_clients.index==clients_id_top_3[0]].sort_values('order_date', ascending=True)
orders_by_client_1498904 = info_orders_top_3_clients[info_orders_top_3_clients.index==clients_id_top_3[1]].sort_values('order_date', ascending=True)
orders_by_client_1491889 = info_orders_top_3_clients[info_orders_top_3_clients.index==clients_id_top_3 [2]].sort_values('order_date', ascending=True)

print('Заказы клиента с id 197929')
display(orders_by_client_197929.head(3))
print('-------------------------------\n')
print('Заказы клиента с id 1498904')
display(orders_by_client_1498904.head(3))
print('-------------------------------\n')
print('Заказы клиента с id 1491889')
display(orders_by_client_1491889.head(3))

In [ ]:
info_orders_top_3_clients.to_csv('info_orders_top_3_clients.csv')

In [ ]:
# Вопрос: каковы даты заказов топ-3 клиентов
info_orders_top_3_clients['year'] = info_orders_top_3_clients['order_date'].dt.year
info_orders_top_3_clients['month'] = info_orders_top_3_clients['order_date'].dt.month
info_orders_top_3_clients['day'] = info_orders_top_3_clients['order_date'].dt.day

info_orders_by_top_3_clients_short = info_orders_top_3_clients[['year', 'day', 'month']]
info_orders_by_top_3_clients_short_pivot = pd.pivot_table(info_orders_by_top_3_clients_short, index=['user_id', 'year', 'month', 'day'])

print("Сведения о заказах топ-3 клиентов\n")
display(info_orders_by_top_3_clients_short_pivot)

In [ ]:
# # График распределения заказов топ-3-клиентов во времени
# plt.plot(data=info_orders_by_top_3_clients_short_pivot)
# plt.show()

In [ ]:
info_orders_by_top_3_clients_short_pivot.to_csv('info_orders_by_top_3_clients_short_pivot.csv')

In [ ]:
# Загружаю датасет order_lines
from google.colab import files
files.upload()

In [ ]:
order_lines_path = '/content/order_lines.csv'
order_lines = pd.read_csv(order_lines_path, sep=';')

print('Сведения о заказах клиентов за все время')
display(order_lines.iloc[:12])

In [ ]:
# Фильтрую таблицу order_lines по идентификатору заказа [order_id]

# orders_by_client_197929 # заказы клиентов с clients_id = 197929
# orders_by_client_1498904 # заказы клиентов с clients_id = 1498904
# orders_by_client_1491889 # заказы клиентов с clients_id = 1491889



list_orders_id_by_client_197929 = list(orders_by_client_197929.order_id.values)
baskets_by_client_197929 = order_lines[order_lines['order_id'].isin(list_orders_id_by_client_197929)]

print('Сведения о составе заказа клиента с user_id = 1491889\n')
display(baskets_by_client_197929.head(3))
print('---------------------------------------------')

list_orders_id_by_client_1491889 = list(orders_by_client_1491889.order_id.values)
baskets_by_client_1491889 = order_lines[order_lines['order_id'].isin(list_orders_id_by_client_1491889)]

print('Сведения о составе заказа клиента с user_id = 1491889\n')
display(baskets_by_client_1491889.head(3))
print('---------------------------------------------')

list_orders_id_by_client_1498904 = list(orders_by_client_1498904.order_id.values)
baskets_by_client_1498904 = order_lines[order_lines['order_id'].isin(list_orders_id_by_client_1498904)]

print('Сведения о составе заказа клиента с user_id = 1498904\n')
display(baskets_by_client_1498904.head(3))

+ График трендов заказов: линейный график для отображения кол-ва заказов по времени
+ Карта: карта с раположением городов и складов, показывающая кол-во заказов в каждом городе
+ Диаграмма распредления заказов по складам: гистограмма, показывающая кол-во заказов по каждому складу
+ Показатель активности пользователей: карта/ диаграмма, показывающая количество уникальныз пользователей в разных городаъ или на разных складах
+ Ключевые метрики на дашборде: KPI с текущим количеством заказов, уникальных пользователей, срежним количеством заказов на пользователя.

# task 2/3

----------
2 задание
----------

Выгрузить список user_id тех пользователей,
кто купил за период 1-15 августа 2 любых корма для животных, кроме "Корм Kitekat для кошек, с кроликом в соусе, 85 г".
Отправить файл .xls



In [ ]:
import pandas as pd

In [ ]:
from  google.colab import drive
drive.mount('/content/drive')

In [ ]:
ord_lines_path = '/content/drive/MyDrive/Technical Interview/Samokat Tech Data Analytics: Jun.Grade/datasets/order_lines.csv'

order_lines = pd.read_csv(ord_lines_path, sep=';')
display(order_lines)
# order_lines.info()

In [ ]:
order_lines['order_id'] = order_lines['order_id'].astype(str)
order_lines.info()

----------
3 задание
----------

Напиши SQL запрос к воображаемой БД, которая имеет структуру как у тебя получилось в 1 задании в Power BI.
Т.е. те же 4 таблицы, те же ключи и те же поля.
Запрос должен вернуть топ 5 самых часто встречающихся товаров в первых заказах пользователей в СПб за период 15-30 августа.

--
После того как сделаешь - отправляй на почту HR.

Удачи :)